In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.preprocessing.text
import keras.backend as K
import pickle

In [ ]:
filePath="/content/drive/MyDrive/classification_selected3/Scrapped_Data.csv"
dataframe = pandas.read_csv(filePath, header=0)
dataframe.drop_duplicates(inplace=True)
data = dataframe
print(dataframe.head())

                    اسم الكاتب  ...                                        ملخص الكتاب
0  كتاب التاريخ الصغير ج 2 PDF  ...  \n\n\n\r\n     (adsbygoogle = window.adsbygoog...
2  كتاب التاريخ الكبير ج 6 PDF  ...  \n\n\n\r\n     (adsbygoogle = window.adsbygoog...
3  كتاب التاريخ الكبير ج 3 PDF  ...  \n\n\n\r\n     (adsbygoogle = window.adsbygoog...
4  كتاب التاريخ الكبير ج 4 PDF  ...  \n\n\n\r\n     (adsbygoogle = window.adsbygoog...
5  كتاب التاريخ الكبير ج 5 PDF  ...  \n\n\n\r\n     (adsbygoogle = window.adsbygoog...

[5 rows x 6 columns]


In [ ]:
print(type(data['اسم الكاتب']))

<class 'pandas.core.series.Series'>


In [ ]:
train_size = int(len(data)*0.8)

In [ ]:
name = data['اسم الكاتب']
category = data['نوع الكتاب']

In [ ]:
train_names = name[:train_size]
train_categories = category[:train_size]

In [ ]:
test_names = name[train_size:]
test_categories = category[train_size:]

In [ ]:
tokenizer = Tokenizer(num_words=None, lower=False)
tokenizer.fit_on_texts(texts) 

In [ ]:
x_train = tokenizer.texts_to_matrix(train_names, mode='tfidf')  # tfidf  ---> every word have weight
x_test = tokenizer.texts_to_matrix(test_names, mode='tfidf')


In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
encoder = LabelEncoder()
encoder.fit(category)
tagst = encoder.fit_transform(category)

In [ ]:
num_classes = int((len(set(tagst))))

In [ ]:
y_train = encoder.fit_transform(train_categories)
y_test = encoder.fit_transform(test_categories)

In [ ]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
max_words = vocab_size

In [ ]:
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val


In [ ]:
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy', 'Recall', 'Precision',
                       f1_metric, 'TruePositives', 'TrueNegatives', 'FalsePositives',
                       'FalseNegatives'])


In [ ]:
batch_size = 5
epochs = 2

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

model.save('my_model.h1')

Epoch 1/2
99/99 [==============================] - 4s 20ms/step - loss: 2.0862 - categorical_accuracy: 0.2955 - recall: 0.4049 - precision: 0.1947 - f1_metric: 0.2733 - true_positives: 200.0000 - true_negatives: 6089.0000 - false_positives: 827.0000 - false_negatives: 294.0000 - val_loss: 6.1174 - val_categorical_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1_metric: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 676.0000 - val_false_positives: 94.0000 - val_false_negatives: 55.0000
Epoch 2/2
99/99 [==============================] - 1s 8ms/step - loss: 1.1430 - categorical_accuracy: 0.6862 - recall: 0.7672 - precision: 0.4422 - f1_metric: 0.5799 - true_positives: 379.0000 - true_negatives: 6438.0000 - false_positives: 478.0000 - false_negatives: 115.0000 - val_loss: 7.4459 - val_categorical_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1_metric: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_tr

In [ ]:
Evaluation_valus = model.evaluate(x_test, y_test, verbose=0)
print("Loss", 'categorical_accuracy', 'Recall', 'Precision',
      'f1_metric', 'TruePositives', 'TrueNegatives', 'FalsePositives',
      'FalseNegatives')

print(Evaluation_valus)

Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[4.000485897064209, 0.0, 0.021739130839705467, 0.021276595070958138, 0.019999990239739418, 3.0, 1794.0, 138.0, 135.0]


In [ ]:
def unique(tags):
    unique_list = []
    un = []
    for x in tags:
        if x not in unique_list:
            unique_list.append(x)

    unique_list.sort()
    return unique_list

In [ ]:
un_list=[]
print("the unique values from 1st list is")
un_list =unique(category)
print(un_list)

the unique values from 1st list is
['\nأحمد تيمور\n', '\nالإمام أبو حامد الغزالي\n', '\nالشيخ يوسف القرضاوي\n', '\nالكاتب أحمد بن عبد الحليم بن عبد السلام بن تيمية\n', '\nالكاتب سلامة موسى\n', '\nالكاتب علي بن نايف الشحود\n', '\nالكاتب محمد بن إسماعيل البخاري\n', '\nالكاتب محمد عمارة\n', '\nالكاتب محمد قطب\n', '\nالكاتب محمود سالم\n', '\nالكاتب هارون يحيى\nالكاتب هارون يحيى\n', '\nالكاتب والت ديزني\n', '\nدكتور محمد حسين هيكل\n', '\nعمرو بن بحر الجاحظ\n', '\nموريس لوبلان\n']


In [ ]:
for x in data["اسم الكاتب"][0:20]:
    tokens = tokenizer.texts_to_matrix([x], mode='tfidf')
    predict_x=model.predict(tokens) 
    classes_x=np.argmax(predict_x,axis=1)
    print(predict_x,"= \t",classes_x,"\t")

[[0.11701342 0.13565472 0.1102711  0.0010142  0.00141031 0.9907602
  0.20095271 0.16809532 0.15946436 0.01980054 0.19851118 0.21425176
  0.00181448 0.00179538 0.00151777]] = 	 [5] 	
[[6.9565982e-02 1.0777938e-01 9.9591255e-02 1.2812018e-03 1.8273294e-03
  9.8877060e-01 2.6766372e-01 1.5858680e-01 1.9867641e-01 2.0876348e-02
  2.2015226e-01 3.2182872e-01 1.6717613e-03 8.9368224e-04 1.2416840e-03]] = 	 [5] 	
[[0.10089561 0.12802836 0.10570839 0.00107393 0.00125253 0.9909345
  0.20608485 0.18925038 0.18353602 0.02587163 0.24137545 0.20107862
  0.00153196 0.001573   0.00108421]] = 	 [5] 	
[[0.09339958 0.12923455 0.09358698 0.00112998 0.00154316 0.98977935
  0.3443358  0.16316661 0.21147922 0.03481078 0.2844209  0.30273706
  0.00225756 0.00169101 0.001212  ]] = 	 [5] 	
[[0.08781704 0.12230787 0.1471267  0.00153264 0.00170124 0.98750293
  0.1925942  0.1792098  0.19754994 0.02608037 0.2745298  0.28148085
  0.00218192 0.00168854 0.00134206]] = 	 [5] 	
[[0.14081845 0.142892   0.09799147 0.00149